In [11]:
# install this package first using the guideline of HSBC : 

#!pip install mlxtend
!pip install -U scikit-learn

  Using cached https://files.pythonhosted.org/packages/70/8e/682770fc1da047bb56443150bfd8d87d850459cd7cc412a5311de3abaa4a/scikit_learn-0.23.1-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/f7/12/ec3f2e203afa394a149911729357aa48affc59c20e2c1c8297a60f33f133/threadpoolctl-2.1.0-py3-none-any.whl
  Found existing installation: scikit-learn 0.20.3
    Uninstalling scikit-learn-0.20.3:
      Successfully uninstalled scikit-learn-0.20.3


In [92]:
# code EDA : Read the data and the industry mapping : 
import pandas as pd
import numpy as np

df_train=pd.read_excel("C:/Users/KINSUK/Desktop/GRAbble/Grablle_short.xlsx",sheet_name="Sheet1")
df_var_class_map=pd.read_excel("C:/Users/KINSUK/Desktop/GRAbble/Grablle_short.xlsx",sheet_name="Var_Name_Class")

# df_var_class_map.head()
df_train.head()

len(df_train)




In [5]:
df_train.describe()
df_dtypes=pd.DataFrame({'Variable' : df_train.dtypes.index, 'Type' : df_train.dtypes.values})

df_dtypes[df_dtypes['Type']=='object']



,Variable,Type
3,Industry,object
4,AUDITMETHOD_1,object
5,AUDITMETHOD_2,object
6,AUDITMETHOD_3,object
7,AUDITMETHOD_4,object


In [2]:
## Missing Value Investigation: 
percent_missing = df_train.isnull().sum() * 100 / len(df_train)
missing_value_df = pd.DataFrame({'column_name': df_train.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True)
missing_value_df
selected_columns_df=missing_value_df[missing_value_df['percent_missing']<=10.0]
selected_col_list=selected_columns_df['column_name'].tolist()
df_train_missing10prct=df_train[selected_col_list]
df_train_missing10prct.head()

,Rec_ID,Total_Liab_to_Total_Assets_1,Equity_to_Assets_1,Equity_to_Liability_1,TOTAL_LIABILITIES_DER_4,TOTAL_LIABILITIES_DER_3,TOTAL_LIABILITIES_DER_2,TOTAL_LIABILITIES_DER_1,EBITDA_to_curr_liab_1,WORKING_CAPITAL_1,...,NET_INCOME_3,EBIT_3,Total_Liab_to_Total_Assets_3,EQUITY_3,EBITDA_to_curr_liab_3,GROSS_PROFIT_3,TOTAL_ASSETS_3,TURNOVER_3,COST_OF_GOODS_SOLD_1,Gross_Margin_perc_1
0,45627,0.6656,0.3344,0.5025,1617338,1617338,1617338,1617338,0.6932,-39811,...,12839.0,52120.0,2.691013,461864.0,0.943090,1911068.0,601014.0,1911068.0,NaN,NaN
1,45628,0.3727,0.6273,1.6829,257405,257405,257405,257405,0.1642,-235493,...,8453.0,12095.0,0.378548,352573.0,0.065361,105040.0,679981.0,122829.0,11753.0,91.3457
2,45629,0.5435,0.4565,0.8398,656246,656246,656246,656246,1.0880,291837,...,138155.0,193010.0,1.044694,267103.0,1.058328,906769.0,628170.0,2778313.0,3420276.0,31.5682
3,45630,0.9093,0.0907,0.0998,8392554,8392554,8392554,8392554,0.3138,-333324,...,72888.0,132111.0,0.978766,1245403.0,0.431591,3026310.0,8574631.0,10350340.0,7905311.0,29.0853
4,45631,1.1082,-0.1082,-0.0976,8619029,8619029,8619029,8619029,-0.7566,-299664,...,-222939.0,-31662.0,1.134244,-1173803.0,-0.040271,631436.0,7598921.0,783822.0,237933.0,69.4446


In [3]:
## Flooring and capping and outlier treatment : 
df=df_train_missing10prct

df.columns
df.columns.to_series().groupby(df.dtypes).groups
num_list=df.select_dtypes(include=['floating','integer']).columns.tolist()
num_list.remove('Rec_ID')
num_list.remove('Target')


numlist_ind=num_list+['Industry']

print(len(df.columns))
print(len(num_list))



197
191


In [4]:
new_list = [
    list(set(df.columns).difference(num_list))
]
new_list



[['AUDITMETHOD_1',
  'AUDITMETHOD_3',
  'Rec_ID',
  'Industry',
  'AUDITMETHOD_2',
  'Target']]

In [9]:
df_percentile = df[numlist_ind].groupby('Industry').quantile([0.01,0.99])
df_percentile.reset_index
df_percentile['Industry']=df_percentile.index.get_level_values(0)
df_percentile['Percentile']=df_percentile.index.get_level_values(1)
df_percentile.head()

Total_Liab_to_Total_Assets_1  Equity_to_Assets_1  \
Industry                                                                     
Agriculture - Dairy 0.01                      0.069970            0.013200   
                    0.99                      0.986800            0.930030   
Agriculture - Other 0.01                      0.055756           -0.192677   
                    0.99                      1.192677            0.943418   
Construction        0.01                      0.268536           -0.707177   

                          Equity_to_Liability_1  TOTAL_LIABILITIES_DER_4  \
Industry                                                                   
Agriculture - Dairy 0.01               0.017600                175890.10   
                    0.99              13.881570               2489349.70   
Agriculture - Other 0.01              -0.124518                 35264.84   
                    0.99              15.948411               9396292.09   
Construction        0.01              -0.395758                574158.04   

                          TOTAL_LIABILITIES_DER_3  TOTAL_LIABILITIES_DER_2  \
Industry                                                                     
Agriculture - Dairy 0.01                175890.10                175890.10   
                    0.99               2489349.70               2489349.70   
Agriculture - Other 0.01                 35264.84                 35264.84   
                    0.99               9396292.09               9396292.09   
Construction        0.01                574158.04                574158.04   

                          TOTAL_LIABILITIES_DER_1  EBITDA_to_curr_liab_1  \
Industry                                                                   
Agriculture - Dairy 0.01                175890.10              -0.103240   
                    0.99               2489349.70               0.807610   
Agriculture - Other 0.01                 35264.84              -0.100853   
                    0.99               9396292.09               3.534171   
Construction        0.01                574158.04               0.030220   

                          WORKING_CAPITAL_1  TOTAL_ASSETS_1  ...  \
Industry                                                     ...   
Agriculture - Dairy 0.01        -2076968.10       647135.20  ...   
                    0.99          526990.50      9527742.10  ...   
Agriculture - Other 0.01        -2073608.25       335181.33  ...   
                    0.99         1417539.80     17326098.82  ...   
Construction        0.01        -3784129.59      1010705.25  ...   

                          Total_Liab_to_Total_Assets_3    EQUITY_3  \
Industry                                                             
Agriculture - Dairy 0.01                      0.085989   -21884.27   
                    0.99                      1.369823  8952974.82   
Agriculture - Other 0.01                      0.050631  -140359.41   
                    0.99                      1.373917  8129115.88   
Construction        0.01                      0.071083     4593.59   

                          EBITDA_to_curr_liab_3  GROSS_PROFIT_3  \
Industry                                                          
Agriculture - Dairy 0.01               0.040033        86688.66   
                    0.99               1.159655       578261.01   
Agriculture - Other 0.01              -1.019195        45937.00   
                    0.99               6.662602      2283458.88   
Construction        0.01              -0.173196        28735.13   

                          TOTAL_ASSETS_3  TURNOVER_3  COST_OF_GOODS_SOLD_1  \
Industry                                                                     
Agriculture - Dairy 0.01       593942.60   128850.78              42691.15   
                    0.99      9129847.81  1187133.74            1336094.09   
Agriculture - Other 0.01       290890.00    62655.73              12255.33   
                    0.99     13609217.

In [14]:
##### 


df.replace([-np.Inf,np.Inf], np.nan,inplace=True)
df_trn_ot= df.groupby(df.Industry.values).transform(lambda x: np.clip(x,x.quantile(0.05),x.quantile(0.95))if x.name in num_list else x)
df_trn_ot.head()


H:\Installed_Files\Anaconda\lib\site-packages\pandas\core\frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


,Rec_ID,Total_Liab_to_Total_Assets_1,Equity_to_Assets_1,Equity_to_Liability_1,TOTAL_LIABILITIES_DER_4,TOTAL_LIABILITIES_DER_3,TOTAL_LIABILITIES_DER_2,TOTAL_LIABILITIES_DER_1,EBITDA_to_curr_liab_1,WORKING_CAPITAL_1,...,NET_INCOME_3,EBIT_3,Total_Liab_to_Total_Assets_3,EQUITY_3,EBITDA_to_curr_liab_3,GROSS_PROFIT_3,TOTAL_ASSETS_3,TURNOVER_3,COST_OF_GOODS_SOLD_1,Gross_Margin_perc_1
0,45627,0.66560,0.33440,0.502500,1617338.00,1617338.00,1617338.00,1617338.00,0.693200,-39811.0,...,12839.0,52120.0,2.397216,461864.00,0.943090,1911068.00,601014.0,1911068.00,NaN,NaN
1,45628,0.37270,0.62730,1.682900,337112.55,337112.55,337112.55,337112.55,0.164200,-235493.0,...,8453.0,12095.0,0.378548,352573.00,0.065361,250716.05,679981.0,288385.95,26741.00,91.3457
2,45629,0.54350,0.45650,0.839800,656246.00,656246.00,656246.00,656246.00,1.088000,291837.0,...,138155.0,193010.0,1.044694,267103.00,1.058328,906769.00,628170.0,2778313.00,3420276.00,31.5682
3,45630,0.88719,0.11281,0.127335,3454562.20,3454562.20,3454562.20,3454562.20,0.313800,-333324.0,...,72888.0,132111.0,0.978766,1245403.00,0.431591,777112.45,7045880.0,2672890.80,2400479.35,29.0853
4,45631,0.98652,0.01348,0.021610,6754168.60,6754168.60,6754168.60,6754168.60,-0.726895,-299664.0,...,-222939.0,-31662.0,1.115603,-394053.25,-0.040271,631436.00,7598921.0,783822.00,237933.00,69.4446


In [97]:
## KNN Imputation : 

from sklearn.impute import KNNImputer
pd.options.mode.chained_assignment = None

imputer = KNNImputer(n_neighbors=3)

#select distinct induystries :
Industry_list=df_train.Industry.unique()

df_trn_ot_mvi_knn=pd.DataFrame()

for i in range(len(Industry_list)):
    df_ind_temp = pd.DataFrame()
    industry= Industry_list[i]
    df_ind_temp = df_trn_ot[df_trn_ot['Industry']==industry]
    df_ind_temp[num_list]=imputer.fit_transform(df_ind_temp[num_list])
    df_trn_ot_mvi_knn = df_trn_ot_mvi_knn.append(df_ind_temp)
 
    df_trn_ot_mvi_knn

,Rec_ID,Total_Liab_to_Total_Assets_1,Equity_to_Assets_1,Equity_to_Liability_1,TOTAL_LIABILITIES_DER_4,TOTAL_LIABILITIES_DER_3,TOTAL_LIABILITIES_DER_2,TOTAL_LIABILITIES_DER_1,EBITDA_to_curr_liab_1,WORKING_CAPITAL_1,...,NET_INCOME_3,EBIT_3,Total_Liab_to_Total_Assets_3,EQUITY_3,EBITDA_to_curr_liab_3,GROSS_PROFIT_3,TOTAL_ASSETS_3,TURNOVER_3,COST_OF_GOODS_SOLD_1,Gross_Margin_perc_1
0,45627,0.66560,0.33440,0.50250,1617338.0,1617338.0,1617338.0,1617338.0,0.69320,-39811.00,...,12839.00,52120.000000,2.397216,4.618640e+05,0.943090,1.911068e+06,6.010140e+05,1.911068e+06,1.437273e+06,32.182133
8,45635,0.37380,0.62620,1.67550,228554.0,228554.0,228554.0,228554.0,-0.05650,374067.00,...,64830.00,72755.333333,0.649784,3.206653e+05,1.022447,3.628771e+05,6.778289e+05,1.302496e+06,1.169189e+06,26.768600
12,45639,0.46460,0.53540,1.15220,1593922.0,1593922.0,1593922.0,1593922.0,2.72200,235876.00,...,590688.00,635271.000000,0.786992,1.265066e+06,5.156453,1.908200e+05,2.025335e+06,6.535789e+05,9.698570e+05,38.956137
14,45641,0.55350,0.44650,0.80670,862221.0,862221.0,862221.0,862221.0,-0.04580,695558.00,...,-271414.50,-298454.800000,0.456316,7.442100e+05,-0.239195,1.830234e+05,1.889525e+06,6.266608e+06,7.093790e+06,8.132520
17,45644,0.82620,0.17380,0.21040,2317750.0,2317750.0,2317750.0,2317750.0,0.20660,242579.00,...,29150.00,32619.000000,2.397216,2.754980e+04,0.090089,2.565270e+05,7.664360e+05,1.221274e+06,6.112829e+06,17.213000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,45786,0.49789,0.50211,1.01009,3522355.0,3522355.0,3522355.0,3522355.0,0.25180,3732874.85,...,703168.60,690379.000000,0.668306,2.752670e+06,0.341800,3.748017e+06,6.082224e+06,3.748017e+06,5.229079e+06,36.152600
196,45823,0.91330,0.08670,0.09500,13160083.0,13160083.0,13160083.0,13160083.0,0.02950,898065.00,...,-179534.00,-160279.000000,1.096354,2.749440e+05,-0.006258,5.145059e+06,1.200349e+07,5.145059e+06,5.229079e+06,36.152600
87,45714,0.77966,0.22034,0.28912,727691.8,727691.8,727691.8,727691.8,0.28139,-195123.00,...,105950.45,180321.850000,0.799422,5.153238e+05,0.618272,7.327391e+05,1.334018e+06,1.549889e+06,6.422757e+05,50.413500
176,45803,0.28709,0.71291,2.77168,2722842.4,2722842.4,2722842.4,2722842.4,0.75749,4541505.00,...,340930.55,563611.150000,0.319080,7.167815e+06,0.635178,3.466779e+06,9.595065e+06,1.040682e+07,7.493157e+06,36.098280


In [81]:
from sklearn.impute import KNNImputer
# from fancyimpute import KNN
pd.options.mode.chained_assignment = None
imputer = KNNImputer(n_neighbors=3)

df_ind_temp = df_trn_ot[df_trn_ot['Industry']=='EduArtsHealthSocial']
# df_ind_temp=pd.DataFrame(imputer.fit_transform(df_ind_temp[num_list]))

df_ind_temp[num_list]=imputer.fit_transform(df_ind_temp[num_list])

df_ind_temp



,Rec_ID,Total_Liab_to_Total_Assets_1,Equity_to_Assets_1,Equity_to_Liability_1,TOTAL_LIABILITIES_DER_4,TOTAL_LIABILITIES_DER_3,TOTAL_LIABILITIES_DER_2,TOTAL_LIABILITIES_DER_1,EBITDA_to_curr_liab_1,WORKING_CAPITAL_1,...,NET_INCOME_3,EBIT_3,Total_Liab_to_Total_Assets_3,EQUITY_3,EBITDA_to_curr_liab_3,GROSS_PROFIT_3,TOTAL_ASSETS_3,TURNOVER_3,COST_OF_GOODS_SOLD_1,Gross_Margin_perc_1
1,45628,0.372700,0.627300,1.68290,337112.55,337112.55,337112.55,337112.55,0.164200,-235493.0,...,8453.00,12095.0,0.378548,352573.00,0.065361,250716.05,679981.00,288385.95,2.674100e+04,91.345700
4,45631,0.986520,0.013480,0.02161,6754168.60,6754168.60,6754168.60,6754168.60,-0.726895,-299664.0,...,-222939.00,-31662.0,1.115603,-394053.25,-0.040271,631436.00,7598921.00,783822.00,2.379330e+05,69.444600
26,45653,0.496500,0.503500,1.01420,3439340.00,3439340.00,3439340.00,3439340.00,0.208900,1344988.0,...,-374586.00,-374558.0,0.795234,3306570.00,0.088564,7812264.00,4324944.00,7984563.00,1.328740e+05,97.945900
29,45656,0.496500,0.503500,1.01420,3439340.00,3439340.00,3439340.00,3439340.00,0.208900,1344988.0,...,-374586.00,-374558.0,0.795234,3306570.00,0.088564,7812264.00,4324944.00,7984563.00,1.328740e+05,97.945900
40,45667,0.921000,0.079000,0.08580,380032.00,380032.00,380032.00,380032.00,3.974300,-37034.0,...,87490.00,104061.0,0.831185,25812.00,7.491690,329157.00,602013.95,377532.00,4.172900e+04,89.649900
50,45677,0.307000,0.693000,2.25690,963829.00,963829.00,963829.00,963829.00,0.302200,-247537.0,...,-652140.00,-576663.0,0.318689,1984415.00,-0.376007,479397.00,3024353.00,1065276.00,5.972120e+05,54.706100
59,45686,0.696500,0.303500,0.43570,2098457.00,2098457.00,2098457.00,2098457.00,0.241700,-920421.0,...,-822676.00,-725673.0,0.685651,405790.00,-0.145166,684868.00,3060532.00,4808972.00,4.723137e+06,24.459500
74,45701,0.271900,0.728100,2.67740,771577.00,771577.00,771577.00,771577.00,5.544750,250279.0,...,1244480.00,1255542.0,0.282818,1948302.00,14.218750,2565011.00,2728176.00,2565011.00,5.781597e+05,54.009733
85,45712,0.696500,0.303500,0.43570,2098457.00,2098457.00,2098457.00,2098457.00,0.241700,-920421.0,...,-822676.00,-725673.0,0.685651,405790.00,-0.145166,684868.00,3060532.00,4808972.00,4.723137e+06,24.459500
102,45729,0.584700,0.415300,0.71030,669240.00,669240.00,669240.00,669240.00,0.279600,-605608.0,...,114391.00,126102.0,0.958833,238730.00,0.285501,668526.00,697973.00,738174.00,5.889700e+04,93.314700


In [11]:
df_trn_ot.iloc[df_trn_ot.values==np.inf] 

,Rec_ID,Total_Liab_to_Total_Assets_1,Equity_to_Assets_1,Equity_to_Liability_1,TOTAL_LIABILITIES_DER_4,TOTAL_LIABILITIES_DER_3,TOTAL_LIABILITIES_DER_2,TOTAL_LIABILITIES_DER_1,EBITDA_to_curr_liab_1,WORKING_CAPITAL_1,...,NET_INCOME_3,EBIT_3,Total_Liab_to_Total_Assets_3,EQUITY_3,EBITDA_to_curr_liab_3,GROSS_PROFIT_3,TOTAL_ASSETS_3,TURNOVER_3,COST_OF_GOODS_SOLD_1,Gross_Margin_perc_1


In [109]:
df[df.Rec_ID.isin(['45679','45709','45797','45672'])]

,Rec_ID,GROSS_PROFIT_1,NET_INCOME_1,NET_INTANGIBLES_1,NPBT_1,RETAINED_EARNINGS_1,SENIOR_NET_DEBT_1,TANGIBLE_NET_WORTH_1,TOTAL_ASSETS_1,TURNOVER_1,...,Debt_to_TNW_1,NetFixedAssets_to_TotalDebt_1,CashFlow_cagr,Cash_to_Total_assets_cagr,DEPRECIATION_2,Cash_to_TO_cagr,TotalDebt_to_TangibleAssets_2,TOTAL_DEBT_2,EBITDA_to_total_Debt_2,Debt_to_TNW_2
45,45672,356686,16892,0,16892,424658,-199446,620318,770992,1416002,...,NaN,NaN,0.3532,0.4080,442.0,0.3838,NaN,NaN,NaN,NaN
52,45679,1610027,439950,0,439950,2498951,-349598,2498951,2642525,4197497,...,0.0109,10.9113,0.4554,0.3923,38402.0,0.5242,NaN,NaN,NaN,NaN
82,45709,166448,-27030,0,-27030,-27030,129195,-26892,188555,402401,...,-5.8817,0.1730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,45797,611290,-57069,0,-48221,338834,1221032,599484,6260036,7776896,...,2.7290,0.0038,-0.0295,-0.4096,2078.0,-0.3321,0.447528,1774220.0,0.087414,2.940332


In [110]:
df_trn_ot[df_trn_ot.Rec_ID.isin(['45679','45709','45797','45672'])]



,Rec_ID,GROSS_PROFIT_1,NET_INCOME_1,NET_INTANGIBLES_1,NPBT_1,RETAINED_EARNINGS_1,SENIOR_NET_DEBT_1,TANGIBLE_NET_WORTH_1,TOTAL_ASSETS_1,TURNOVER_1,...,Debt_to_TNW_1,NetFixedAssets_to_TotalDebt_1,CashFlow_cagr,Cash_to_Total_assets_cagr,DEPRECIATION_2,Cash_to_TO_cagr,TotalDebt_to_TangibleAssets_2,TOTAL_DEBT_2,EBITDA_to_total_Debt_2,Debt_to_TNW_2
45,45672,356686.0,16892.0,0.0,16892.0,424658.0,-199446.0,620318.00,770992.00,1416002.00,...,NaN,NaN,0.3532,0.4080,936.6,0.3838,NaN,NaN,NaN,NaN
52,45679,1610027.0,439950.0,0.0,439950.0,2498951.0,-349598.0,2498951.00,2642525.00,4197497.00,...,0.01090,5.947500,0.4554,0.3923,38402.0,0.5242,NaN,NaN,NaN,NaN
82,45709,166448.0,-27030.0,0.0,-27030.0,-27030.0,129195.0,-9596.55,419333.95,471317.75,...,-0.14741,0.173000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,45797,611290.0,-57069.0,0.0,-48221.0,338834.0,1221032.0,599484.00,6260036.00,7776896.00,...,2.72900,0.012205,-0.0295,-0.4096,2078.0,-0.3321,0.447528,1774220.0,0.087414,2.940332


In [10]:
df_trn_ot.CashFlow_cagr[df_trn_ot['Rec_ID']==45646]

19   NaN
Name: CashFlow_cagr, dtype: float64

In [12]:
## MVI by inudstry wise median values :
# df_trn_ot_mvi = df_trn_ot.apply(lambda x: x.str.strip()).replace('', np.nan)
# df_trn_ot_mvi=df_trn_ot.groupby('Industry').apply(lambda x: x.fillna(x.mean())if x.name in num_list else x)

# x['val']=x.groupby('user')['val'].apply(lambda x: x.fillna(x.interpolate()))

df_trn_ot_mvi = df_trn_ot.fillna(df_trn_ot.groupby('Industry')[num_list].transform('median'))
# df_trn_ot_mvi['AUDITMETHOD_2'] = df_trn_ot_mvi['AUDITMETHOD_2'].fillna(df_trn_ot_mvi.groupby('Industry')['AUDITMETHOD_2'].transform('mode'))

df_check=df_trn_ot_mvi.isnull().sum()
df_check2=pd.DataFrame({'Variable':df_check.index, '#Missing_Values':df_check.values})

# df_check.to_csv("C:/Users/45018883/Desktop/GRABBLE/df_check.csv", sep='\t')
df_check2
df_check2[df_check2['#Missing_Values']>0]
df_trn_ot_mvi.iloc[df_trn_ot_mvi.values==np.inf] 



,Rec_ID,Total_Liab_to_Total_Assets_1,Equity_to_Assets_1,Equity_to_Liability_1,TOTAL_LIABILITIES_DER_4,TOTAL_LIABILITIES_DER_3,TOTAL_LIABILITIES_DER_2,TOTAL_LIABILITIES_DER_1,EBITDA_to_curr_liab_1,WORKING_CAPITAL_1,...,NET_INCOME_3,EBIT_3,Total_Liab_to_Total_Assets_3,EQUITY_3,EBITDA_to_curr_liab_3,GROSS_PROFIT_3,TOTAL_ASSETS_3,TURNOVER_3,COST_OF_GOODS_SOLD_1,Gross_Margin_perc_1


In [12]:
df_check2[df_check2['#Missing_Values']>0]
# df_check2

,Variable,#Missing_Values
69,AUDITMETHOD_2,920


In [ ]:
#Standardize the variables :
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()

X_train = sc_X.fit_transform(X_train)

In [13]:
##********************* Variable wise AR alculation (Don't Run this) ######################************************:


from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

#Standardize the variables :
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()

# df_trn_ot_mvi = df_trn_ot_mvi.reset_index()
# from sklearn.cross_validation import  train_test_split
# xtrain,xtest,ytrain,ytest  = train_test_split(x,y,random_state=2)
# ytrain=np.ravel(ytrain)

y_train= df_trn_ot_mvi['Target'].astype(int).values
x_train=df_trn_ot_mvi['Cash_to_TO_cagr'].values.reshape(-1, 1)

X_train = sc_X.fit_transform(X_train)

y_train=np.ravel(y_train)

clf = LogisticRegression(solver='liblinear')
clf.fit(x_train,y_train)
ypred_train = clf.predict(x_train)
# accuracy_score(y_train,ypred_train)
AR =accuracy_score(y_train,ypred_train)
print(AR)

print(confusion_matrix(y_train,ypred_train))


0.9750989881214255
[[33246     0]
 [  849     0]]


In [13]:
## Variable wise AR alculation under loop for all variablese :


from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler


sc_X = StandardScaler()
y_train= df_trn_ot_mvi['Target'].astype(int).values
y_train=np.ravel(y_train)
AR_Df=  pd.DataFrame(columns=['Variable', 'Accuracy_Ratio'])



for i in range(len(num_list)):
    Accu_Rat=[]
    var_name=[]
    x_train=[]
    ypred_train=[]
    clf=[]
    
    x_train = sc_X.fit_transform(df_trn_ot_mvi[num_list[i]].values.reshape(-1, 1))
    var_name=num_list[i]
    
    clf = LogisticRegression(solver='liblinear')
    clf.fit(x_train,y_train)
    ypred_train = clf.predict(x_train)
    Accu_Rat =accuracy_score(y_train,ypred_train)
    AR_Df=AR_Df.append(pd.DataFrame({'Variable' : var_name , 'Accuracy_Ratio' : Accu_Rat},index=[0]),ignore_index=True)


AR_Df_2= pd.merge(AR_Df,df_var_class_map, on='Variable', how='inner')
AR_Df_2


NameError: name 'df_trn_ot_mvi' is not defined

In [15]:
#Select top variables of each class : 

AR_Df_3=AR_Df_2.groupby('Variable_Class',as_index=False).max()
# AR_Df_3=AR_Df_2.groupby('Variable_Class',as_index=False).agg({'Variable':'min','Accuracy_Ratio' :'max'})
# df.groupby(['name'], as_index=False).agg({'value1': 'sum', 'value2': 'sum', 'otherstuff1': 'first', 'otherstuff2': 'first'})
AR_Df_3.sort_values('Accuracy_Ratio')
final_var_list=AR_Df_3['Variable'].tolist()
x_train_final= df_trn_ot_mvi[final_var_list]
x_train_final
x_train_final.isnull().sum()

CASH_AND_CASH_EQUIVALENTS_floored4    0
COST_OF_GOODS_SOLD_1                  0
CURRENT_ASSETS_3                      0
CURRENT_LIABILITIES_3                 0
CashFlow_2_1                          0
                                     ..
Total_Liab_to_Total_Assets_cagr       0
WC_to_TO_3                            0
WC_to_TO_2_1                          0
WC_to_TO_cagr                         0
WORKING_CAPITAL_3                     0
Length: 107, dtype: int64

In [21]:
## Stepwise feature selection selection :

y_train= df_trn_ot_mvi['Target']

from mlxtend.feature_selection import SequentialFeatureSelector as sfs

clf = LogisticRegression(solver='liblinear',max_iter=2000,dual=False)

# Build step forward feature selection
sfs1 = sfs(clf,k_features=(1,107),forward=True,floating=False, scoring='accuracy',cv=0)

# Perform SFFS
sfs1 = sfs1.fit(x_train_final, y_train)

sfs1.k_feature_names_

x_train_sfs=sfs1.k_feature_names_



In [30]:
len(x_train_sfs)
x_train_sfs= df_trn_ot_mvi[list(x_train_sfs)]

In [31]:
#############************** Variable wise AR alculation using RFE  (To Be Updated) ***********************###################  :

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression


logreg = LogisticRegression(solver='liblinear',max_iter=2000)
rfe = RFE(logreg, n_features_to_select=1)

# y_train= df_trn_ot_mvi['Target'].astype(int).values.reshape(-1, 1)
y_train= df_trn_ot_mvi['Target']
rfe = rfe.fit(x_train_sfs,y_train)

order_rfe=rfe.ranking_
order_rfe

array([36, 37, 32, 27, 34, 10, 11,  3, 30,  4,  9, 16, 15,  1,  8, 18, 35,
       39, 25, 23, 17, 21, 14, 33, 20, 13, 28,  7,  5,  2, 31, 24, 22, 29,
        6, 26, 19, 12, 38])

In [45]:
print(rfe.support_)
print(order_rfe)

feature_ranks = pd.DataFrame(columns=['Rank', 'Variable'])

for i in order_rfe :
    Rank=[]
    Variable=[]
    
#     Rank= rfe.get_support(i)
    Rank=i
    Variable=x_train_final.columns[i-1]
    feature_ranks=feature_ranks.append(pd.DataFrame({'Rank' : Rank , 'Variable' : Variable},index=[0]),ignore_index=True)
    
feature_ranks
feature_ranks.sort_values("Rank",ascending=True).head()

##Select the first 35 variables :
x_train_rfe_list = feature_ranks[feature_ranks['Rank']<=35]['Variable']
len(x_train_rfe_list)
x_train_rfe_df=x_train_sfs= df_trn_ot_mvi[x_train_rfe_list]


[False False False False False False False False False False False False
 False  True False False False False False False False False False False
 False False False False False False False False False False False False
 False False False]
[36 37 32 27 34 10 11  3 30  4  9 16 15  1  8 18 35 39 25 23 17 21 14 33
 20 13 28  7  5  2 31 24 22 29  6 26 19 12 38]


In [50]:
## Let's fit the first decision tree model : 

from sklearn.tree import DecisionTreeClassifier

# Make a decision tree and train
tree = DecisionTreeClassifier()
tree.fit(x_train_rfe_df, y_train)
print(f'Model Accuracy: {tree.score(x_train_rfe_df, y_train)}')
tree.tree_.__getstate__()

Model Accuracy: 1.0


{'max_depth': 6,
 'node_count': 21,
 'nodes': array([( 1,  4, 18, -6.73497486e+00, 0.04875   , 200, 200.),
        ( 2,  3, 34,  5.90491062e-03, 0.5       ,   2,   2.),
        (-1, -1, -2, -2.00000000e+00, 0.        ,   1,   1.),
        (-1, -1, -2, -2.00000000e+00, 0.        ,   1,   1.),
        ( 5,  8, 32, -2.80210996e+00, 0.0395878 , 198, 198.),
        ( 6,  7, 20, -1.79592749e+00, 0.5       ,   2,   2.),
        (-1, -1, -2, -2.00000000e+00, 0.        ,   1,   1.),
        (-1, -1, -2, -2.00000000e+00, 0.        ,   1,   1.),
        ( 9, 12, 19,  2.35546464e-02, 0.03014369, 196, 196.),
        (10, 11,  2,  3.17894801e-01, 0.5       ,   2,   2.),
        (-1, -1, -2, -2.00000000e+00, 0.        ,   1,   1.),
        (-1, -1, -2, -2.00000000e+00, 0.        ,   1,   1.),
        (13, 16,  8, -8.11899275e-01, 0.02040599, 194, 194.),
        (14, 15, 20,  1.35065001e+00, 0.24489796,   7,   7.),
        (-1, -1, -2, -2.00000000e+00, 0.        ,   6,   6.),
        (-1, -1, -2, -2.0

In [52]:
## We will now fit Random forest :

from sklearn.ensemble import RandomForestClassifier

# Create the model with 100 trees
model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True)
# Fit on training data
model_rf= model.fit(x_train_rfe_df, y_train)
rf_score= model_rf.score(x_train_rfe_df, y_train)
rf_score

1.0